In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
lemm = WordNetLemmatizer()

lines=100000

In [2]:

def create_lexicon():
    lexicon=[]
    with open('pos.txt','r') as f:
        contents = f.readlines()
        #all_words=word_tokenize(contents)
        for l in contents[:lines]:
            all_words=word_tokenize(l)
            lexicon+=list(all_words)

                
                    
            
    with open('neg.txt','r') as f:
        contents = f.readlines()
        for l in contents[:lines]:
            all_words=word_tokenize(l)
            lexicon += list(all_words)
    
    lexicon = [lemm.lemmatize(i,pos='n') for i in lexicon]
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        #print(w_counts[w])
        if 1000 > w_counts[w] > 50:
            l2.append(w)
    return l2

In [3]:
def make_features(sample,lexicon,char):
    import numpy as np
    featureset=[]
    with open(sample,'r')as f:
        contents=f.readlines()
        
        for l in contents[:lines]:
            sam_words=word_tokenize(l)
            sam_words=[lemm.lemmatize(i) for i in sam_words]
            features=np.zeros(len(lexicon))
            for word in sam_words:
                if word.lower() in lexicon:
                    index_value=lexicon.index(word.lower())
                    features[index_value]+=1
            features=list(features)
            featureset.append([features,char])
    return featureset

In [31]:
import pandas as pd
data=pd.read_csv(r"C:\Users\PIYUSH\Desktop\news.csv")

In [32]:
len(data["headline"].unique())

2873

In [33]:
news=data["text"]
headline=data["headline"]

In [34]:
def data_prepare(headline):
    lexicon=[]
    for i in headline:
        all_words=word_tokenize(i)
        lexicon+=list(all_words)
    head_lexicon=[]
    stop = set(stopwords.words('english'))
    head_lexicon+=[i for i in lexicon if i not in stop]
    head_lexicon = [lemm.lemmatize(i,pos='n') for i in head_lexicon]
    head_words=[word.lower() for word in head_lexicon if word.isalpha()]
    return head_words

In [37]:
head_words=data_prepare(headline)
head_lexicons=set(head_words)

In [38]:
len(head_lexicons)

5396